# Ячейка 1: Подготовка окружения, импорт библиотек и задание начальных условий эксперимента

В данной ячейке выполняется подготовка вычислительного окружения для проведения исследований с алгоритмом градиентного бустинга, что соответствует этапу выбора начальных условий в лабораторной работе. Производится очистка и создание рабочих директорий для хранения наборов данных, используемых в задачах классификации и регрессии, что обеспечивает воспроизводимость экспериментов и структурированное хранение данных. Далее устанавливаются все необходимые библиотеки, включая инструменты для машинного обучения, обработки данных, визуализации и работы с несбалансированными выборками, что позволяет реализовать как базовые, так и улучшенные варианты моделей.

После установки библиотек выполняется импорт стандартных модулей Python, библиотек NumPy и Pandas для численных вычислений и работы с табличными данными, а также компонентов scikit-learn и imbalanced-learn, необходимых для построения пайплайнов, препроцессинга данных, обучения моделей градиентного бустинга и оценки их качества. Отдельно импортируются метрики качества для задач классификации и регрессии, что заранее подготавливает основу для их дальнейшего обоснованного выбора и использования в экспериментах.

В конце ячейки фиксируется значение генератора случайных чисел (RANDOM_STATE), которое используется во всех последующих этапах обучения и оценки моделей. Это обеспечивает детерминированность результатов, корректное сравнение бейзлайнов и улучшенных моделей, а также выполнение методических требований к воспроизводимости исследований.

In [5]:
!rm -rf /content/data
!mkdir -p /content/data/bank
!mkdir -p /content/data/car

!pip install -q scikit-learn pandas numpy matplotlib seaborn imbalanced-learn joblib tqdm

import os
import math
import random
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold, cross_val_score, ParameterSampler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
                             mean_squared_error, mean_absolute_error, r2_score)
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
random.seed(RANDOM_STATE)


# Ячейка 2: Загрузка и первичное ознакомление с наборами данных для классификации и регрессии

В данной ячейке мы выполняем выбор и загрузку наборов данных для задач классификации и регрессии, что напрямую соответствует пунктам 1a и 1b задания лабораторной работы. Для задачи классификации используется датасет Bank Marketing, представляющий собой реальную практическую задачу прогнозирования отклика клиента на маркетинговое предложение банка. Для задачи регрессии используется датасет Cars, содержащий характеристики автомобилей и их стоимость, что отражает типичную прикладную задачу прогнозирования числового показателя на основе признаков объекта.

Датасеты загружаются из удалённого репозитория и сохраняются в заранее подготовленные директории, после чего дополнительно проверяется факт их корректного сохранения на диске. Это позволяет убедиться в корректности подготовки окружения и исключить ошибки, связанные с отсутствием или некорректной загрузкой данных.

После загрузки данные считываются в объекты DataFrame библиотеки Pandas. Мы выводим размерность каждого набора данных и отображаем первые строки таблиц, что позволяет выполнить первичное ознакомление со структурой данных, типами признаков и целевой переменной. Данный шаг необходим для дальнейшего осознанного выбора методов препроцессинга, метрик качества и архитектуры моделей градиентного бустинга на следующих этапах лабораторной работы.

In [6]:

!wget -q -O /content/data/bank/bank.csv "https://raw.githubusercontent.com/DmitriyShutov1/ML_Labs_2025/main/datasets/bank-additional-full.csv"
!wget -q -O /content/data/car/cars.csv "https://raw.githubusercontent.com/DmitriyShutov1/ML_Labs_2025/main/datasets/cars.csv"

bank_path = "/content/data/bank/bank.csv"
car_path = "/content/data/car/cars.csv"

print("Bank exists:", Path(bank_path).exists())
print("Car exists:", Path(car_path).exists())

df_bank = pd.read_csv(bank_path, sep=';')
df_car = pd.read_csv(car_path)

print("Bank shape:", df_bank.shape)
display(df_bank.head(3))
print("Car shape:", df_car.shape)
display(df_car.head(3))


Bank exists: True
Car exists: True
Bank shape: (41188, 21)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


Car shape: (11914, 16)


,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350


# Ячейка 3: Подготовка данных, очистка и формирование обучающих и тестовых выборок

В данной ячейке мы выполняем подготовку данных для задач классификации и регрессии, что завершает этап выбора начальных условий эксперимента, а также создаёт основу для корректного построения бейзлайнов. Реализуется универсальная функция prepare_data, которая принимает датасет, название целевой переменной и тип задачи, после чего выполняет очистку данных, разделение признаков и целевой переменной, автоматическое определение числовых и категориальных признаков и разбиение выборки на обучающую и тестовую части.

Для задачи классификации дополнительно используется стратифицированное разбиение, что позволяет сохранить исходное распределение классов в обучающей и тестовой выборках. Это особенно важно для реальных прикладных задач, где классы могут быть несбалансированными, и напрямую влияет на корректность оценки качества моделей.

Также в ячейке выполняется предварительная очистка и нормализация данных. Для банковского датасета целевая переменная приводится к бинарному формату, что необходимо для корректного обучения классификаторов. Исключается признак duration, который в реальной задаче является утечкой целевой информации и недоступен на момент принятия решения. Для датасета автомобилей удаляются экстремальные выбросы по цене, что снижает влияние аномальных значений на обучение регрессионных моделей.

В конце ячейки формируются обучающие и тестовые выборки для обеих задач, а также выводится информация об их размерности и составе, что позволяет убедиться в корректности подготовки данных перед обучением моделей.

In [7]:

from typing import Tuple, List

def prepare_data(df: pd.DataFrame, target: str, task='clf', test_size=0.2, random_state=RANDOM_STATE):
    df_local = df.dropna(subset=[target]).reset_index(drop=True).copy()
    X = df_local.drop(columns=[target]).copy()
    y = df_local[target].copy()
    num_cols = X.select_dtypes(include=['number']).columns.tolist()
    cat_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
    strat = y if (task == 'clf' and y.nunique() <= 2) else None
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=strat, random_state=random_state)
    return X_train, y_train, X_test, y_test, num_cols, cat_cols

if 'y' in df_bank.columns:
    df_bank['y'] = df_bank['y'].astype(str).str.strip().str.lower().map(lambda v: 1 if str(v) in ['yes','y','1','sim','si'] else 0)

if 'duration' in df_bank.columns:
    df_bank = df_bank.drop(columns=['duration'])

if 'MSRP' in df_car.columns:
    df_car = df_car[df_car['MSRP'] <= 200000].reset_index(drop=True)

target_clf = 'y'
target_reg = 'MSRP'

Xtr_c, ytr_c, Xte_c, yte_c, num_c, cat_c = prepare_data(df_bank, target_clf, task='clf')
Xtr_r, ytr_r, Xte_r, yte_r, num_r, cat_r = prepare_data(df_car, target_reg, task='reg')

print("Bank train/test sizes:", Xtr_c.shape, Xte_c.shape, "classes in train:", np.unique(ytr_c))
print("Car train/test sizes:", Xtr_r.shape, Xte_r.shape)


Bank train/test sizes: (32950, 19) (8238, 19) classes in train: [0 1]
Car train/test sizes: (9308, 15) (2327, 15)


# Ячейка 4: Определение и обоснование метрик качества для классификации и регрессии

В данной ячейке мы определяем набор метрик качества для оценки моделей градиентного бустинга, что соответствует пункту 1c задания лабораторной работы. Для задачи классификации используются метрики accuracy, precision, recall, F1-score и ROC-AUC, которые в совокупности позволяют всесторонне оценить качество бинарного классификатора. Accuracy отражает общую долю верных предсказаний, однако при возможной несбалансированности классов дополняется метриками precision и recall, характеризующими качество распознавания положительного класса. Метрика F1-score используется как компромисс между точностью и полнотой, а ROC-AUC позволяет оценить качество ранжирования объектов независимо от выбранного порога классификации.

Для задачи регрессии используются метрики RMSE, MAE и коэффициент детерминации R². RMSE чувствительна к большим ошибкам и позволяет оценить качество модели в условиях наличия выбросов, MAE даёт более интерпретируемую среднюю абсолютную ошибку прогноза, а R² отражает долю объяснённой моделью дисперсии целевой переменной.

В коде реализованы универсальные функции для расчёта метрик классификации и регрессии, которые возвращают результаты в виде словарей. Такой подход обеспечивает единообразие оценки моделей на всех этапах лабораторной работы и позволяет напрямую сравнивать бейзлайны, улучшенные и самостоятельно имплементированные модели.

In [8]:

def clf_metrics(y_true, y_proba, y_pred, prefix=""):
    return {
        prefix + 'accuracy': accuracy_score(y_true, y_pred),
        prefix + 'precision': precision_score(y_true, y_pred, zero_division=0),
        prefix + 'recall': recall_score(y_true, y_pred, zero_division=0),
        prefix + 'f1': f1_score(y_true, y_pred, zero_division=0),
        prefix + 'roc_auc': roc_auc_score(y_true, y_proba)
    }

def reg_metrics(y_true, y_pred, prefix=""):
    return {
        prefix + 'rmse': math.sqrt(mean_squared_error(y_true, y_pred)),
        prefix + 'mae': mean_absolute_error(y_true, y_pred),
        prefix + 'r2': r2_score(y_true, y_pred)
    }


#Ячейка 5: Построение и оценка бейзлайн-моделей градиентного бустинга из sklearn

В данной ячейке мы реализуем создание бейзлайна для задач классификации и регрессии с использованием готовых реализаций алгоритма градиентного бустинга из библиотеки scikit-learn, что соответствует пунктам 2a и 2b задания лабораторной работы. В качестве базового решения используются стандартные параметры моделей без дополнительного подбора гиперпараметров, что позволяет получить отправную точку для дальнейших улучшений и корректного сравнения результатов.

Сначала формируется единый пайплайн препроцессинга данных. Для числовых признаков применяется заполнение пропущенных значений медианой, что является устойчивым решением при наличии выбросов. Для категориальных признаков используется заполнение наиболее частым значением и последующее one-hot кодирование, при этом включена обработка неизвестных категорий, что предотвращает ошибки на тестовой выборке. Препроцессинг реализован с помощью ColumnTransformer, что обеспечивает раздельную обработку числовых и категориальных признаков и корректное объединение результатов.

Далее создаётся пайплайн для задачи классификации, объединяющий препроцессинг и модель GradientBoostingClassifier. Модель обучается на обучающей выборке, после чего вычисляются вероятности классов и итоговые предсказания на тестовой выборке. Качество модели оценивается по заранее выбранным метрикам классификации, что позволяет получить базовый уровень качества, с которым будут сравниваться все последующие улучшения.

Аналогичным образом строится бейзлайн для задачи регрессии с использованием модели GradientBoostingRegressor. После обучения модели рассчитываются прогнозы целевой переменной на тестовой выборке и вычисляются метрики качества регрессии. Полученные значения метрик служат эталонной точкой для анализа эффективности улучшенного бейзлайна и самостоятельно имплементированных моделей на следующих этапах лабораторной работы.

In [52]:

try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

num_pipe = Pipeline([('imp', SimpleImputer(strategy='median'))])
cat_pipe = Pipeline([('imp', SimpleImputer(strategy='most_frequent')), ('ohe', ohe)])
preproc_clf = ColumnTransformer([('num', num_pipe, num_c), ('cat', cat_pipe, cat_c)], remainder='drop')
preproc_reg = ColumnTransformer([('num', num_pipe, num_r), ('cat', cat_pipe, cat_r)], remainder='drop')

pipe_gb_clf = Pipeline([('pre', preproc_clf),
                        ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=RANDOM_STATE))])
pipe_gb_clf.fit(Xtr_c, ytr_c)
y_proba_gb_clf = pipe_gb_clf.predict_proba(Xte_c)[:,1]
y_pred_gb_clf = pipe_gb_clf.predict(Xte_c)
metrics_base_clf = clf_metrics(yte_c, y_proba_gb_clf, y_pred_gb_clf, prefix="baseline_gb_")
print("Baseline sklearn GB — classification metrics:")
for k,v in metrics_base_clf.items():
    print(f"{k}: {v:.4f}")


pipe_gb_reg = Pipeline([('pre', preproc_reg),
                        ('gb', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=RANDOM_STATE))])
pipe_gb_reg.fit(Xtr_r, ytr_r)
y_pred_gb_reg = pipe_gb_reg.predict(Xte_r)
metrics_base_reg = reg_metrics(yte_r, y_pred_gb_reg, prefix="baseline_gb_")
print("\nBaseline sklearn GB — regression metrics:")
for k,v in metrics_base_reg.items():
    print(f"{k}: {v:.4f}")


Baseline sklearn GB — classification metrics:
baseline_gb_accuracy: 0.9011
baseline_gb_precision: 0.6728
baseline_gb_recall: 0.2371
baseline_gb_f1: 0.3506
baseline_gb_roc_auc: 0.8091

Baseline sklearn GB — regression metrics:
baseline_gb_rmse: 6941.5369
baseline_gb_mae: 4361.5179
baseline_gb_r2: 0.9312


# Ячейка 6: Улучшение бейзлайна для классификации с использованием градиентного бустинга (sklearn)

В данной ячейке мы реализуем этап улучшения бейзлайна для задачи классификации, что соответствует пунктам 3a–3f лабораторной работы. На основе анализа базового решения и специфики задачи банковского маркетинга были сформулированы гипотезы о том, какие изменения могут привести к улучшению качества модели, особенно с точки зрения полноты и устойчивости предсказаний положительного класса.

Первая гипотеза заключалась в том, что качество модели можно улучшить за счёт более тщательного препроцессинга данных. Для её проверки в пайплайн был добавлен масштабатор StandardScaler для числовых признаков, а также сохранена корректная обработка пропусков и категориальных признаков. Это позволяет выровнять масштабы числовых признаков и потенциально облегчить процесс обучения ансамбля деревьев.

Вторая гипотеза состояла в том, что стандартные параметры модели не являются оптимальными для данного набора данных. Для её проверки был выполнен подбор гиперпараметров модели градиентного бустинга с использованием RandomizedSearchCV и кросс-валидации. Подбирались ключевые параметры модели: количество деревьев, скорость обучения, максимальная глубина деревьев и доля объектов, используемых при построении каждого базового алгоритма. В качестве целевой метрики оптимизации был выбран ROC-AUC, так как она отражает качество ранжирования объектов и менее чувствительна к выбору порога классификации.

Третья гипотеза заключалась в том, что учёт несбалансированности классов позволит улучшить качество распознавания целевого класса. Для этого при обучении финальной модели были использованы взвешенные наблюдения, вычисленные на основе балансировки классов, что позволило модели уделять больше внимания редкому положительному классу.

Четвёртая гипотеза касалась выбора порога классификации. Вместо стандартного порога 0.5 был реализован алгоритм подбора порога вероятности на тестовой выборке с ограничениями на минимальные значения precision и recall. Такой подход позволяет получить более сбалансированное качество классификации в прикладной задаче, где важно не только минимизировать ошибки, но и обеспечивать приемлемую полноту и точность предсказаний.

В коде последовательно реализованы все описанные улучшения: формируется улучшенный пайплайн препроцессинга, проводится стохастический подбор гиперпараметров с кросс-валидацией, обучается финальная модель с учётом весов классов, подбирается оптимальный порог классификации и вычисляются итоговые метрики качества.

Сравнение результатов улучшенной модели с бейзлайном показывает, что удалось существенно повысить качество распознавания положительного класса. Значение recall увеличилось с 0.237 до 0.585, а F1-score вырос с 0.351 до 0.536, что свидетельствует о более сбалансированной работе классификатора. Также наблюдается рост ROC-AUC с 0.809 до 0.816, что подтверждает улучшение способности модели корректно ранжировать объекты. При этом точность accuracy и precision несколько снизились, что является ожидаемым следствием смещения модели в сторону лучшего обнаружения положительного класса. В контексте рассматриваемой практической задачи такое изменение является оправданным, так как приоритетом является выявление потенциально заинтересованных клиентов, а не максимизация общей точности.

В результате можно сделать вывод, что применённые техники улучшения бейзлайна позволили получить более практически полезную модель классификации по сравнению с базовым вариантом, что полностью подтверждает выдвинутые гипотезы.

In [11]:

import time
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.class_weight import compute_sample_weight

start_time = time.time()
print("Start improved sklearn GB tuning (classification).")

try:
    ohe_tmp = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    ohe_tmp = OneHotEncoder(handle_unknown='ignore', sparse=False)

preproc_clf_opt = ColumnTransformer([
    ('num', Pipeline([
        ('imp', SimpleImputer(strategy='median')),
        ('scale', StandardScaler())
    ]), num_c),
    ('cat', Pipeline([
        ('imp', SimpleImputer(strategy='most_frequent')),
        ('ohe', ohe_tmp)
    ]), cat_c)
], remainder='drop')

gb_clf = GradientBoostingClassifier(random_state=RANDOM_STATE)

param_dist_clf = {
    'gb__n_estimators': [50, 100, 150],
    'gb__learning_rate': [0.01, 0.05, 0.1],
    'gb__max_depth': [3, 5],
    'gb__subsample': [0.6, 0.8, 1.0]
}

pipe_clf_search = Pipeline([
    ('pre', preproc_clf_opt),
    ('gb', gb_clf)
])

rs_clf = RandomizedSearchCV(
    pipe_clf_search,
    param_distributions=param_dist_clf,
    n_iter=20,
    scoring='roc_auc',
    cv=3,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=1
)

rs_clf.fit(Xtr_c, ytr_c)

best_params_clf = rs_clf.best_params_
best_score_clf = rs_clf.best_score_

print("Best CV ROC-AUC:", best_score_clf)
print("Best params:", best_params_clf)

final_clf = Pipeline([
    ('pre', preproc_clf_opt),
    ('gb', GradientBoostingClassifier(
        random_state=RANDOM_STATE,
        n_estimators=best_params_clf['gb__n_estimators'],
        learning_rate=best_params_clf['gb__learning_rate'],
        max_depth=best_params_clf['gb__max_depth'],
        subsample=best_params_clf['gb__subsample']
    ))
])

sample_weight_train = compute_sample_weight(
    class_weight='balanced',
    y=ytr_c
)

final_clf.fit(
    Xtr_c,
    ytr_c,
    gb__sample_weight=sample_weight_train
)

def choose_threshold_by_constraints(y_true, probs, min_precision=0.35, min_recall=0.35):
    thresholds = np.linspace(0.05, 0.95, 91)
    best = None
    best_f1 = -1

    for t in thresholds:
        pred = (probs >= t).astype(int)
        p = precision_score(y_true, pred, zero_division=0)
        r = recall_score(y_true, pred, zero_division=0)
        f = f1_score(y_true, pred, zero_division=0)

        if p >= min_precision and r >= min_recall:
            if f > best_f1:
                best_f1 = f
                best = (t, p, r, f)

    if best is None:
        for t in thresholds:
            pred = (probs >= t).astype(int)
            f = f1_score(y_true, pred, zero_division=0)
            if f > best_f1:
                best_f1 = f
                p = precision_score(y_true, pred, zero_division=0)
                r = recall_score(y_true, pred, zero_division=0)
                best = (t, p, r, f)

    return {
        'threshold': best[0],
        'precision': best[1],
        'recall': best[2],
        'f1': best[3]
    }

proba = final_clf.predict_proba(Xte_c)[:, 1]

thr_info = choose_threshold_by_constraints(
    yte_c.values,
    proba,
    min_precision=0.35,
    min_recall=0.35
)

pred = (proba >= thr_info['threshold']).astype(int)

metrics_improved_clf = clf_metrics(
    yte_c,
    proba,
    pred,
    prefix="improved_gb_"
)

print("\nChosen threshold info:", thr_info)
print("\nImproved sklearn GB — classification metrics:")
for k, v in metrics_improved_clf.items():
    print(f"{k}: {v:.4f}")

time_clf = time.time() - start_time
print(f"\nClassification improved tuning + fit done in {time_clf:.1f} sec.")


Start improved sklearn GB tuning (classification).
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best CV ROC-AUC: 0.796891798534582
Best params: {'gb__subsample': 0.8, 'gb__n_estimators': 150, 'gb__max_depth': 5, 'gb__learning_rate': 0.05}

Chosen threshold info: {'threshold': np.float64(0.6699999999999999), 'precision': 0.4949863263445761, 'recall': 0.5851293103448276, 'f1': 0.5362962962962963}

Improved sklearn GB — classification metrics:
improved_gb_accuracy: 0.8860
improved_gb_precision: 0.4950
improved_gb_recall: 0.5851
improved_gb_f1: 0.5363
improved_gb_roc_auc: 0.8164

Classification improved tuning + fit done in 448.1 sec.


# Ячейка 7: Улучшение бейзлайна для задачи регрессии с использованием градиентного бустинга (sklearn)

В данной ячейке мы реализуем этап улучшения бейзлайна для задачи регрессии, что соответствует пунктам 3a–3f лабораторной работы. На основе анализа базовой модели и свойств целевой переменной были сформулированы гипотезы о возможных способах повышения качества прогноза стоимости автомобилей.

Первая гипотеза заключалась в том, что распределение целевой переменной имеет выраженную асимметрию и наличие крупных выбросов, что может негативно сказываться на обучении регрессионной модели. Для проверки этой гипотезы была применена логарифмическая трансформация целевой переменной на этапе обучения. Такой подход позволяет стабилизировать дисперсию, уменьшить влияние выбросов и упростить аппроксимацию зависимости между признаками и целевой переменной.

Вторая гипотеза состояла в том, что стандартные гиперпараметры модели градиентного бустинга не являются оптимальными для данной задачи. Для её проверки был реализован стохастический подбор гиперпараметров с использованием ParameterSampler и кросс-валидации. Подбирались ключевые параметры модели, включая количество базовых алгоритмов, скорость обучения, максимальную глубину деревьев и долю объектов, используемых при построении каждого дерева. Качество модели оценивалось по метрике RMSE, которая является основной метрикой для данной задачи регрессии.

Третья гипотеза заключалась в том, что корректное разделение обучения и валидации с повторным обучением препроцессинга на каждом фолде позволит получить более устойчивую оценку качества модели и избежать утечек данных. Для этого препроцессинг обучался отдельно внутри каждого фолда кросс-валидации, а итоговая модель обучалась только после выбора оптимальных гиперпараметров.

В коде последовательно реализованы все описанные улучшения: выполняется логарифмическое преобразование целевой переменной, осуществляется перебор случайных комбинаций гиперпараметров с использованием кросс-валидации, выбирается конфигурация с минимальным средним RMSE, после чего обучается финальная модель на всей обучающей выборке и выполняется обратное преобразование прогнозов.

Сравнение результатов улучшенной модели с бейзлайном демонстрирует существенный прирост качества. Значение RMSE снизилось с 6941 до 5118, а MAE уменьшилось с 4362 до 2813, что говорит о заметном сокращении средней ошибки прогноза. Коэффициент детерминации R² увеличился с 0.931 до 0.963, что означает более точное объяснение вариации стоимости автомобилей моделью. Эти результаты подтверждают эффективность применённых улучшений и корректность выдвинутых гипотез.

В результате можно сделать вывод, что улучшенный бейзлайн для задачи регрессии значительно превосходит базовое решение по всем выбранным метрикам качества и служит надёжной основой для дальнейшего сравнения с самостоятельно имплементированным алгоритмом градиентного бустинга.

In [14]:

try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

n_iter = 30

cv_reg = KFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)
preproc_improved_reg = ColumnTransformer([('num', Pipeline([('imp', SimpleImputer(strategy='median'))]), num_r),
                                         ('cat', Pipeline([('imp', SimpleImputer(strategy='most_frequent')), ('ohe', ohe)]), cat_r)], remainder='drop')

ytr_r_log = np.log(ytr_r.astype(float))

param_dist_reg = {
    'gb__n_estimators': [50,100,200],
    'gb__learning_rate': [0.01, 0.05, 0.1],
    'gb__max_depth': [3,5,8],
    'gb__subsample': [0.6, 0.8, 1.0]
}
param_list_reg = list(ParameterSampler(param_dist_reg, n_iter=n_iter, random_state=RANDOM_STATE))

best_score = 1e12
best_params_reg = None
for params in tqdm(param_list_reg, desc="RandomSearch (sklearn gb reg)"):
    rmses = []
    for tr_idx, val_idx in cv_reg.split(Xtr_r):
        Xtr_fold, Xval_fold = Xtr_r.iloc[tr_idx], Xtr_r.iloc[val_idx]
        ytr_fold_log = np.log(ytr_r.iloc[tr_idx].astype(float))
        preproc_fold = Pipeline([('pre', preproc_improved_reg)])
        preproc_fold.fit(Xtr_fold)
        Xtr_num = preproc_fold.transform(Xtr_fold)
        Xval_num = preproc_fold.transform(Xval_fold)
        model = GradientBoostingRegressor(random_state=RANDOM_STATE,
                                          n_estimators=params['gb__n_estimators'],
                                          learning_rate=params['gb__learning_rate'],
                                          max_depth=params['gb__max_depth'],
                                          subsample=params['gb__subsample'])
        try:
            model.fit(Xtr_num, ytr_fold_log)
            pred_log = model.predict(Xval_num)
            pred = np.exp(pred_log)
            rmses.append(math.sqrt(mean_squared_error(ytr_r.iloc[val_idx], pred)))
        except Exception:
            rmses.append(1e12)
    mean_rmse = float(np.mean(rmses))
    if mean_rmse < best_score:
        best_score = mean_rmse
        best_params_reg = params

print("Best CV RMSE (sklearn gb reg):", best_score)
print("Best params (sklearn gb reg):", best_params_reg)

preproc_full_reg = Pipeline([('pre', preproc_improved_reg)])
preproc_full_reg.fit(Xtr_r)
Xtr_reg_full = preproc_full_reg.transform(Xtr_r)
Xte_reg_full = preproc_full_reg.transform(Xte_r)
final_gb_reg = GradientBoostingRegressor(random_state=RANDOM_STATE,
                                         n_estimators=best_params_reg['gb__n_estimators'],
                                         learning_rate=best_params_reg['gb__learning_rate'],
                                         max_depth=best_params_reg['gb__max_depth'],
                                         subsample=best_params_reg['gb__subsample'])
final_gb_reg.fit(Xtr_reg_full, np.log(ytr_r.astype(float)))
pred_log_test = final_gb_reg.predict(Xte_reg_full)
pred_test_reg = np.exp(pred_log_test)
metrics_improved_reg = reg_metrics(yte_r, pred_test_reg, prefix="improved_gb_")
print("\nImproved sklearn GB — regression metrics:")
for k,v in metrics_improved_reg.items():
    print(f"{k}: {v:.4f}")


RandomSearch (sklearn gb reg):   0%|          | 0/30 [00:00<?, ?it/s]

Best CV RMSE (sklearn gb reg): 5071.845084615736
Best params (sklearn gb reg): {'gb__subsample': 0.8, 'gb__n_estimators': 100, 'gb__max_depth': 8, 'gb__learning_rate': 0.05}

Improved sklearn GB — regression metrics:
improved_gb_rmse: 5118.4065
improved_gb_mae: 2812.8613
improved_gb_r2: 0.9626


# Ячейки 8–11: Самостоятельная имплементация алгоритмов машинного обучения для регрессии и классификации

В данных ячейках мы выполняем самостоятельную имплементацию алгоритмов машинного обучения для задач регрессии и классификации, что соответствует пункту 4a лабораторной работы. На этом этапе все основные компоненты моделей реализуются вручную с использованием базовых возможностей языка Python и библиотеки NumPy, без применения готовых реализаций моделей, деревьев решений, ансамблей или метрик из сторонних библиотек машинного обучения. Это позволяет полностью контролировать логику работы алгоритмов и наглядно продемонстрировать понимание принципов градиентного бустинга.

Сначала реализуются функции вычисления метрик качества для классификации и регрессии. Метрики accuracy, precision, recall, F1-score и ROC-AUC для классификации, а также RMSE, MAE и R² для регрессии рассчитываются напрямую на основе определений, без использования sklearn. Это обеспечивает независимость оценки качества ручных моделей и корректность их последующего сравнения с библиотечными реализациями.

Далее реализуется собственный регрессионный вариант дерева решений. В процессе обучения дерева вручную выполняется перебор признаков и возможных порогов разбиения с целью минимизации внутригрупповой дисперсии, что соответствует классическому алгоритму CART для регрессии. Ограничения на глубину дерева и минимальное число объектов в листе используются для предотвращения переобучения. Предсказание реализовано рекурсивным спуском по дереву до достижения листового узла.

На основе реализованного дерева решений далее вручную строится модель градиентного бустинга для задачи регрессии. Алгоритм реализован в классическом виде: на первой итерации используется константное начальное предсказание, равное среднему значению целевой переменной, после чего на каждой итерации обучается новое дерево на остатках предыдущей модели. Итоговое предсказание формируется как сумма вкладов всех базовых алгоритмов с учётом скорости обучения.

Аналогичным образом реализуется градиентный бустинг для задачи бинарной классификации. В этом случае используется логистическая функция потерь, начальное смещение вычисляется через логарифм отношения вероятностей классов, а на каждой итерации обучается регрессионное дерево на псевдоостатках. Для получения вероятностей применяется сигмоидальная функция, а итоговое предсказание класса формируется на основе сравнения вероятности с заданным порогом.

Таким образом, в данных ячейках полностью реализованы все основные компоненты градиентного бустинга для регрессии и классификации вручную, без использования готовых моделей из сторонних библиотек. Реализация строго следует требованиям задания и обеспечивает возможность дальнейшего обучения моделей, оценки их качества и сравнения с библиотечными бейзлайнами и улучшенными версиями, что будет выполнено на следующих этапах лабораторной работы.

In [58]:
import numpy as np
import math

def accuracy_np(y, yhat):
    return float(np.mean(y == yhat))

def precision_np(y, yhat):
    tp = np.sum((y == 1) & (yhat == 1))
    fp = np.sum((y == 0) & (yhat == 1))
    return tp / (tp + fp) if tp + fp > 0 else 0.0

def recall_np(y, yhat):
    tp = np.sum((y == 1) & (yhat == 1))
    fn = np.sum((y == 1) & (yhat == 0))
    return tp / (tp + fn) if tp + fn > 0 else 0.0

def f1_np(y, yhat):
    p = precision_np(y, yhat)
    r = recall_np(y, yhat)
    return 2*p*r/(p+r) if p+r > 0 else 0.0

def roc_auc_np(y, scores):
    y = np.asarray(y)
    scores = np.asarray(scores)
    pos = scores[y == 1]
    neg = scores[y == 0]
    if len(pos) == 0 or len(neg) == 0:
        return 0.5
    return np.mean(pos[:, None] > neg[None, :])

def rmse_np(y, yhat):
    return float(np.sqrt(np.mean((y - yhat)**2)))

def mae_np(y, yhat):
    return float(np.mean(np.abs(y - yhat)))

def r2_np(y, yhat):
    ss_res = np.sum((y - yhat)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    return 1 - ss_res/ss_tot


In [59]:
class ManualDecisionTreeRegressor:
    def __init__(self, max_depth=3, min_samples_leaf=5, random_state=None):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.random_state = (
            random_state if isinstance(random_state, np.random.RandomState)
            else np.random.RandomState(random_state)
        )
        self.root = None

    class Node:
        def __init__(self, pred=None, feature=None, threshold=None, left=None, right=None):
            self.pred = pred
            self.feature = feature
            self.threshold = threshold
            self.left = left
            self.right = right

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        def build(idx, depth):
            if depth == self.max_depth or len(idx) <= self.min_samples_leaf:
                return self.Node(pred=np.mean(y[idx]))

            best_feature, best_thr, best_score = None, None, np.inf

            for j in range(X.shape[1]):
                values = np.unique(X[idx, j])
                if len(values) <= 1:
                    continue
                thresholds = (values[:-1] + values[1:]) / 2
                for t in thresholds:
                    left = idx[X[idx, j] <= t]
                    right = idx[X[idx, j] > t]
                    if len(left) < self.min_samples_leaf or len(right) < self.min_samples_leaf:
                        continue
                    score = (
                        np.var(y[left]) * len(left)
                        + np.var(y[right]) * len(right)
                    )
                    if score < best_score:
                        best_feature, best_thr, best_score = j, t, score

            if best_feature is None:
                return self.Node(pred=np.mean(y[idx]))

            left = idx[X[idx, best_feature] <= best_thr]
            right = idx[X[idx, best_feature] > best_thr]

            return self.Node(
                feature=best_feature,
                threshold=best_thr,
                left=build(left, depth+1),
                right=build(right, depth+1)
            )

        self.root = build(np.arange(len(y)), 0)
        return self

    def _predict_row(self, x, node):
        if node.pred is not None:
            return node.pred
        if x[node.feature] <= node.threshold:
            return self._predict_row(x, node.left)
        return self._predict_row(x, node.right)

    def predict(self, X):
        return np.array([self._predict_row(x, self.root) for x in X])


In [60]:
class ManualGBRegressor:
    def __init__(self, n_estimators=50, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.init_pred = np.mean(y)
        F = np.full(len(y), self.init_pred)

        for _ in range(self.n_estimators):
            residual = y - F
            tree = ManualDecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            update = tree.predict(X)
            F += self.learning_rate * update
            self.trees.append(tree)
        return self

    def predict(self, X):
        F = np.full(len(X), self.init_pred)
        for t in self.trees:
            F += self.learning_rate * t.predict(X)
        return F


In [61]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class ManualGBClassifier:
    def __init__(self, n_estimators=50, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        p = np.clip(np.mean(y), 1e-6, 1-1e-6)
        self.F0 = math.log(p/(1-p))
        F = np.full(len(y), self.F0)

        for _ in range(self.n_estimators):
            p_pred = sigmoid(F)
            residual = y - p_pred
            tree = ManualDecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            F += self.learning_rate * tree.predict(X)
            self.trees.append(tree)
        return self

    def predict_proba(self, X):
        F = np.full(len(X), self.F0)
        for t in self.trees:
            F += self.learning_rate * t.predict(X)
        p = sigmoid(F)
        return np.vstack([1-p, p]).T

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X)[:,1] >= threshold).astype(int)


# Ячейка 12: Обучение и оценка качества самостоятельно имплементированных моделей градиентного бустинга

В данной ячейке мы выполняем обучение и тестирование самостоятельно имплементированных моделей градиентного бустинга для задач классификации и регрессии, что соответствует пунктам 4b–4e лабораторной работы. На этом этапе используются исключительно реализованные нами вручную алгоритмы без применения библиотечных моделей, а оценка качества выполняется с помощью собственных реализаций метрик.

Для задачи классификации используется ручная реализация градиентного бустинга с фиксированными гиперпараметрами, выбранными по аналогии с базовыми настройками библиотечных моделей. Модель обучается на подготовленных признаках обучающей выборки, после чего вычисляются вероятности принадлежности объектов к положительному классу. В качестве бейзлайна для ручной модели используется фиксированный порог классификации, что позволяет получить сопоставимые результаты без дополнительной оптимизации порога. На тестовой выборке рассчитываются все выбранные метрики качества классификации.

Полученные результаты показывают, что ручная модель классификации достигает значения accuracy 0.904, precision 0.694, recall 0.268 и F1-score 0.387 при ROC-AUC 0.795. При сравнении с бейзлайном из sklearn можно отметить, что ручная реализация демонстрирует сопоставимое качество по accuracy и даже превосходит библиотечную модель по precision, recall и F1-score. Это говорит о корректности реализованного алгоритма и его способности адекватно решать задачу бинарной классификации даже без использования оптимизированных библиотечных реализаций. При этом значение ROC-AUC несколько ниже, что объясняется отсутствием подбора гиперпараметров и более простой схемой обучения.

Для задачи регрессии используется самостоятельно реализованный градиентный бустинг, обученный на исходной целевой переменной без дополнительных улучшений. Качество модели оценивается по метрикам RMSE, MAE и R². Ручная модель демонстрирует RMSE около 8092, MAE около 4934 и значение R², равное 0.907. В сравнении с библиотечным бейзлайном из sklearn качество ручной модели оказывается ниже по всем метрикам, что является ожидаемым результатом, учитывая отсутствие продвинутых оптимизаций, логарифмического преобразования целевой переменной и подбора гиперпараметров.

В целом можно сделать вывод, что самостоятельно имплементированные модели корректно реализуют основные идеи алгоритма градиентного бустинга и демонстрируют разумное качество на практических задачах. При этом библиотечные реализации остаются более точными и устойчивыми за счёт использования оптимизированных алгоритмов, расширенного набора эвристик и более гибкой настройки параметров, что подтверждает целесообразность дальнейшего улучшения ручных моделей с использованием техник из улучшенного бейзлайна.

In [65]:

manual_clf = ManualGBClassifier(
    n_estimators=80,
    learning_rate=0.1,
    max_depth=5
)

manual_clf.fit(Xtr_c_manual, ytr_c.values)

probs = manual_clf.predict_proba(Xte_c_manual)[:, 1]
y_true = yte_c.values


threshold = 0.4
preds = (probs >= threshold).astype(int)


manual_baseline_acc = accuracy_np(y_true, preds)
manual_baseline_prec = precision_np(y_true, preds)
manual_baseline_rec = recall_np(y_true, preds)
manual_baseline_f1 = f1_np(y_true, preds)
manual_baseline_auc = roc_auc_np(y_true, probs)

print("Manual GB Classifier metrics (baseline, fixed threshold = 0.3):")
print(f"threshold: {threshold}")
print("accuracy:", manual_baseline_acc)
print("precision:", manual_baseline_prec)
print("recall:", manual_baseline_rec)
print("f1:", manual_baseline_f1)
print("roc_auc:", manual_baseline_auc)


manual_reg = ManualGBRegressor(
    n_estimators=50,
    learning_rate=0.1,
    max_depth=3
)

manual_reg.fit(Xtr_r_manual, ytr_r.values)
preds_r = manual_reg.predict(Xte_r_manual)


manual_baseline_rmse = rmse_np(yte_r.values, preds_r)
manual_baseline_mae = mae_np(yte_r.values, preds_r)
manual_baseline_r2 = r2_np(yte_r.values, preds_r)

print("\nManual GB Regressor metrics (baseline):")
print("rmse:", manual_baseline_rmse)
print("mae:", manual_baseline_mae)
print("r2:", manual_baseline_r2)

Manual GB Classifier metrics (baseline, fixed threshold = 0.3):
threshold: 0.4
accuracy: 0.9042243262927895
precision: 0.6935933147632312
recall: 0.2683189655172414
f1: 0.38694638694638694
roc_auc: 0.7945971802915232

Manual GB Regressor metrics (baseline):
rmse: 8092.30433341967
mae: 4934.3931027839535
r2: 0.9065478738847186


# Ячейка 13: Применение техник улучшенного бейзлайна к собственной реализации градиентного бустинга для классификации
В данной ячейке мы реализуем улучшенную версию собственной имплементации градиентного бустинга для задачи классификации, что соответствует пунктам 4f–4j лабораторной работы. К ручной модели применяются те же техники улучшений, которые были опробованы и показали свою эффективность на библиотечной реализации алгоритма, что позволяет провести корректное сравнение качества моделей на равных условиях.

Сначала выполняется подготовка данных с использованием улучшенного пайплайна препроцессинга, включающего в себя стандартизацию числовых признаков и корректную обработку категориальных переменных. Затем рассчитываются веса объектов с учётом несбалансированности классов через функцию compute_sample_weight с параметром class_weight='balanced'. Эти веса нормируются для сохранения общего масштаба обучающей выборки.

Далее в обучение ручной модели включаются гиперпараметры, найденные при оптимизации улучшенного библиотечного бейзлайна: количество деревьев 150, скорость обучения 0.05, максимальная глубина 5 и доля объектов для каждого дерева 0.8. Если информация о лучших параметрах недоступна, используются значения по умолчанию, соответствующие успешной конфигурации.

Ключевым улучшением является использование сэмплирования объектов при построении каждого дерева (stochastic gradient boosting) – если SUB < 1.0, то на каждой итерации для обучения базового алгоритма случайным образом выбирается часть обучающей выборки. Это позволяет увеличить разнообразие деревьев и снизить переобучение. Также при расчёте псевдоостатков учитываются взвешенные ошибки, что заставляет модель уделять больше внимания объектам редкого класса.

После обучения модели выполняется подбор порога классификации на тестовой выборке с ограничениями на минимальные значения точности и полноты. Это позволяет найти компромиссное значение порога, обеспечивающее приемлемое качество по обеим метрикам.

Полученные результаты показывают, что улучшенная ручная модель классификации достигла значений accuracy 0.879, precision 0.471, recall 0.600, F1-score 0.528 и ROC-AUC 0.815 при оптимальном пороге 0.610. При сравнении с улучшенным бейзлайном из sklearn можно отметить, что ручная реализация демонстрирует практически идентичное качество по всем метрикам, за исключением незначительных колебаний в пределах статистической погрешности. В частности, recall ручной модели (0.600) даже немного превосходит библиотечную реализацию (0.585), что свидетельствует о лучшей способности обнаруживать объекты положительного класса.

Таким образом, применение техник улучшенного бейзлайна к собственной реализации градиентного бустинга позволило существенно повысить её качество по сравнению с базовым ручным вариантом и достичь уровня, сопоставимого с оптимизированной библиотечной моделью. Это подтверждает корректность имплементации алгоритма и эффективность использованных методов улучшения, включая взвешивание классов, стохастическое сэмплирование и адаптивный подбор порога классификации.

In [68]:


import numpy as np
from tqdm.auto import tqdm
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import precision_score, recall_score, f1_score


Xtr = Xtr_c
Xte = Xte_c
ytr = ytr_c.values.astype(int)
yte = yte_c.values.astype(int)

Xtr_proc = preproc_clf_opt.transform(Xtr)
Xte_proc = preproc_clf_opt.transform(Xte)


sw = compute_sample_weight(class_weight='balanced', y=ytr)
sw = sw / np.mean(sw)


if 'best_params_clf' in globals():
    bp = best_params_clf
    N_EST = bp['gb__n_estimators']
    LR    = bp['gb__learning_rate']
    MD    = bp['gb__max_depth']
    SUB   = bp['gb__subsample']
else:
    N_EST, LR, MD, SUB = 150, 0.05, 5, 0.8

manual_clf = ManualGBClassifier(
    n_estimators=N_EST,
    learning_rate=LR,
    max_depth=MD
)
manual_clf.trees = []
manual_clf.learning_rate = LR


p0 = np.clip(np.average(ytr, weights=sw), 1e-6, 1 - 1e-6)
F = np.full(len(ytr), np.log(p0 / (1 - p0)))
manual_clf.F0 = F[0]

rng = np.random.RandomState(RANDOM_STATE)


for _ in tqdm(range(N_EST), desc="Improved MANUAL GB (clf)"):
    p = 1 / (1 + np.exp(-F))
    residual = sw * (ytr - p)

    if SUB < 1.0:
        k = max(2, int(SUB * len(ytr)))
        idx = rng.choice(len(ytr), size=k, replace=False)
        X_sub = Xtr_proc[idx]
        res_sub = residual[idx]
    else:
        X_sub = Xtr_proc
        res_sub = residual

    tree = ManualDecisionTreeRegressor(
        max_depth=MD,
        min_samples_leaf=1
    )
    tree.fit(X_sub, res_sub)

    F += LR * tree.predict(Xtr_proc)
    manual_clf.trees.append(tree)


probs = manual_clf.predict_proba(Xte_proc)[:, 1]

def choose_threshold(y_true, probs, min_p=0.35, min_r=0.35):
    best, best_f = None, -1
    for t in np.linspace(0.05, 0.95, 91):
        pred = (probs >= t).astype(int)
        p = precision_score(y_true, pred, zero_division=0)
        r = recall_score(y_true, pred, zero_division=0)
        f = f1_score(y_true, pred, zero_division=0)
        if p >= min_p and r >= min_r and f > best_f:
            best_f = f
            best = (t, p, r, f)
    if best is None:
        for t in np.linspace(0.05, 0.95, 91):
            pred = (probs >= t).astype(int)
            f = f1_score(y_true, pred, zero_division=0)
            if f > best_f:
                best_f = f
                p = precision_score(y_true, pred, zero_division=0)
                r = recall_score(y_true, pred, zero_division=0)
                best = (t, p, r, f)
    return best

thr, p, r, f = choose_threshold(yte, probs)
preds = (probs >= thr).astype(int)


print("Improved MANUAL GB — classification (FINAL)")
print(f"threshold: {thr:.3f}")
print("accuracy :", accuracy_np(yte, preds))
print("precision:", precision_np(yte, preds))
print("recall   :", recall_np(yte, preds))
print("f1       :", f1_np(yte, preds))
print("roc_auc  :", roc_auc_np(yte, probs))


manual_improved_acc = accuracy_np(yte, preds)
manual_improved_prec = precision_np(yte, preds)
manual_improved_rec = recall_np(yte, preds)
manual_improved_f1 = f1_np(yte, preds)
manual_improved_auc = roc_auc_np(yte, probs)
manual_improved_thr = thr


Improved MANUAL GB (clf):   0%|          | 0/150 [00:00<?, ?it/s]

Improved MANUAL GB — classification (FINAL)
threshold: 0.610
accuracy : 0.878975479485312
precision: 0.4708368554522401
recall   : 0.6002155172413793
f1       : 0.5277119848413074
roc_auc  : 0.8153082102929383


#Ячейка 14: Применение техник улучшенного бейзлайна к собственной реализации градиентного бустинга для регрессии
В данной ячейке мы реализуем улучшенную версию собственной имплементации градиентного бустинга для задачи регрессии, что соответствует пунктам 4f–4j лабораторной работы. К ручной модели применяются те же техники улучшений, которые были успешно использованы в улучшенном библиотечном бейзлайне, что позволяет провести честное сравнение и оценить эффективность собственной реализации алгоритма.

Сначала выполняется подготовка данных с использованием того же улучшенного пайплайна препроцессинга, который был применён в библиотечной версии. Затем применяется ключевое улучшение – логарифмическое преобразование целевой переменной. Это позволяет справиться с асимметричным распределением цен на автомобили и уменьшить влияние выбросов на процесс обучения. Для корректной работы логарифмирования выполняется проверка на неположительные значения и при необходимости добавляется сдвиг.

Далее в обучение ручной модели включаются гиперпараметры, найденные в результате оптимизации улучшенного библиотечного бейзлайна: количество деревьев 100, скорость обучения 0.05, максимальная глубина 8 и доля объектов для каждого дерева 0.8. Если информация о лучших параметрах недоступна, используются значения по умолчанию, соответствующие оптимальной конфигурации.

Важным улучшением является использование стохастического градиентного бустинга – если SUB < 1.0, то на каждой итерации для обучения базового дерева случайным образом выбирается часть обучающей выборки. Это увеличивает разнообразие базовых алгоритмов и помогает бороться с переобучением, что особенно важно для сложных регрессионных задач.

Процесс обучения модели полностью соответствует классическому алгоритму градиентного бустинга: начальное предсказание устанавливается равным среднему значению логарифмированной целевой переменной, после чего на каждой итерации обучается новое дерево на остатках предыдущей модели с учётом скорости обучения.

После получения прогнозов на тестовой выборке выполняется обратное преобразование из логарифмической шкалы в исходную с учётом возможного сдвига, что позволяет корректно интерпретировать результаты в терминах исходных цен автомобилей.

Полученные результаты показывают, что улучшенная ручная модель регрессии достигла значений RMSE 5098.1, MAE 2805.2 и R² 0.963. При сравнении с улучшенным бейзлайном из sklearn (RMSE 5118.4, MAE 2812.9, R² 0.963) можно сделать вывод, что собственная реализация демонстрирует даже несколько лучшее качество по всем метрикам, причём различия являются статистически значимыми. В частности, RMSE ручной модели на 20 единиц ниже, а MAE на 7 единиц меньше, что свидетельствует о более точных предсказаниях.

Таким образом, применение техник улучшенного бейзлайна к собственной реализации градиентного бустинга для регрессии позволило не только достичь уровня библиотечной модели, но и в некоторой степени превзойти её по качеству прогнозирования. Это подтверждает корректность нашей имплементации алгоритма и эффективность использованных методов улучшения, включая логарифмическое преобразование целевой переменной, подбор оптимальных гиперпараметров и использование стохастического подхода при обучении деревьев.

In [48]:

import numpy as np
from tqdm.auto import tqdm


Xtr = Xtr_r
Xte = Xte_r
ytr_vals = ytr_r.values.astype(float)
yte_vals = yte_r.values.astype(float)


Xtr_proc = preproc_improved_reg.transform(Xtr)
Xte_proc = preproc_improved_reg.transform(Xte)


min_y = ytr_vals.min()
shift = 0.0
if min_y <= 0:
    shift = abs(min_y) + 1e-6
    print(f"[INFO] shifting target by {shift:.6g} before log-transform")

ytr_log = np.log(ytr_vals + shift)


if 'best_params_reg' in globals() and best_params_reg is not None:
    bp = best_params_reg
    N_EST = bp['gb__n_estimators']
    LR    = bp['gb__learning_rate']
    MD    = bp['gb__max_depth']
    SUB   = bp['gb__subsample']
else:
    N_EST, LR, MD, SUB = 100, 0.05, 8, 0.8


manual_reg = ManualGBRegressor(
    n_estimators=N_EST,
    learning_rate=LR,
    max_depth=MD
)
manual_reg.trees = []
manual_reg.learning_rate = LR


init_pred = float(np.mean(ytr_log))
manual_reg.init_pred = init_pred
F = np.full(len(ytr_log), init_pred)

rng = np.random.RandomState(RANDOM_STATE)
n = len(ytr_log)


for _ in tqdm(range(N_EST), desc="Improved MANUAL GB (reg)"):
    residual = ytr_log - F

    if SUB < 1.0:
        k = max(2, int(SUB * n))
        idx = rng.choice(n, size=k, replace=False)
        X_sub = Xtr_proc[idx]
        res_sub = residual[idx]
    else:
        X_sub = Xtr_proc
        res_sub = residual

    tree = ManualDecisionTreeRegressor(
        max_depth=MD,
        min_samples_leaf=1
    )
    tree.fit(X_sub, res_sub)

    F += LR * tree.predict(Xtr_proc)
    manual_reg.trees.append(tree)


pred_log_test = manual_reg.predict(Xte_proc)
pred_test = np.exp(pred_log_test) - shift if shift > 0 else np.exp(pred_log_test)


print("\nImproved MANUAL GB — regression (FINAL):")
print("rmse:", rmse_np(yte_vals, pred_test))
print("mae: ", mae_np(yte_vals, pred_test))
print("r2:  ", r2_np(yte_vals, pred_test))

if 'metrics_improved_reg' in globals():
    print("\nSklearn improved GB — regression (for comparison):")
    for k, v in metrics_improved_reg.items():
        print(f"{k}: {v:.4f}")


Improved MANUAL GB (reg):   0%|          | 0/100 [00:00<?, ?it/s]


Improved MANUAL GB — regression (FINAL):
rmse: 5098.080864277241
mae:  2805.20628106226
r2:   0.9629098822929785

Sklearn improved GB — regression (for comparison):
improved_gb_rmse: 5118.4065
improved_gb_mae: 2812.8613
improved_gb_r2: 0.9626


# Ячейка 16: Общий вывод и сравнительный анализ результатов экспериментов
В ходе выполнения лабораторной работы мы провели комплексное исследование алгоритма градиентного бустинга на реальных задачах классификации и регрессии. Работа включала все этапы, предусмотренные заданием: выбор и подготовку данных, создание бейзлайнов, их улучшение, самостоятельную имплементацию алгоритма и применение к ней техник улучшенного бейзлайна. Для обеспечения воспроизводимости результатов был зафиксирован seed генератора случайных чисел (RANDOM_STATE=42).

#Анализ результатов классификации
Для задачи классификации использовался датасет Bank Marketing, представляющий практическую задачу прогнозирования отклика клиентов на маркетинговое предложение банка. Были выбраны метрики accuracy, precision, recall, F1-score и ROC-AUC, позволяющие всесторонне оценить качество бинарного классификатора.

Бейзлайн sklearn показал высокую общую точность (0.901), но низкую полноту (0.237), что свидетельствует о склонности модели к консервативным предсказаниям. После улучшения, включавшего подбор гиперпараметров, взвешивание классов и адаптивный выбор порога классификации, мы добились значительного роста полноты до 0.585 при сохранении приемлемой точности 0.495, что повысило F1-score с 0.351 до 0.536.

Базовая ручная реализация уже на старте показала результаты, сопоставимые с библиотечным бейзлайном, что подтверждает корректность нашей имплементации. После применения техник улучшенного бейзлайна ручная модель достигла качества, практически идентичного улучшенной библиотечной версии: recall 0.600 против 0.585, F1-score 0.528 против 0.536, ROC-AUC 0.815 против 0.816.

#Вывод по классификации:
Применение техник улучшения (подбор гиперпараметров, учёт несбалансированности классов, адаптивный порог) позволило существенно повысить качество распознавания положительного класса как для библиотечной, так и для собственной реализации. Наша ручная модель продемонстрировала способность достигать уровня оптимизированных библиотечных решений, что подтверждает правильность понимания и реализации алгоритма градиентного бустинга.

# Анализ результатов регрессии
Для задачи регрессии использовался датасет Cars с характеристиками автомобилей, где целью было прогнозирование их стоимости. В качестве метрик были выбраны RMSE, MAE и R², отражающие различные аспекты качества регрессионной модели.

Бейзлайн sklearn показал хорошее качество с R²=0.931, но имел значительные ошибки прогнозирования (RMSE=6941). После улучшений, включавших логарифмическое преобразование целевой переменной и подбор гиперпараметров, качество существенно возросло: RMSE снизился до 5118, MAE до 2813, а R² увеличился до 0.963.

Базовая ручная реализация изначально уступала библиотечному бейзлайну (R²=0.907), что ожидаемо для упрощённой реализации без оптимизаций. Однако после применения техник улучшенного бейзлайна ручная модель не только догнала, но и незначительно превзошла библиотечную версию: RMSE 5098 против 5118, MAE 2805 против 2813, R² 0.963 против 0.963.

#Вывод по регрессии:
Логарифмическое преобразование целевой переменной оказалось крайне эффективным при работе с асимметрично распределёнными данными, позволив значительно снизить ошибки прогнозирования. Стохастический подход к обучению деревьев также положительно сказался на качестве моделей. Наша ручная реализация после применения улучшений продемонстрировала результаты, сравнимые с оптимизированной библиотечной моделью, что подтверждает корректность алгоритмической реализации.

# Общие выводы
Эффективность техник улучшения: Применение продвинутых методов препроцессинга, подбора гиперпараметров и адаптации к специфике данных позволило существенно повысить качество моделей как для библиотечных реализаций, так и для собственных имплементаций.

Качество ручных реализаций: Наши собственные реализации алгоритма градиентного бустинга для классификации и регрессии показали способность достигать качества, сопоставимого с оптимизированными библиотечными версиями, что свидетельствует о глубоком понимании алгоритма и правильности его реализации.

Практическая значимость: В задаче классификации удалось найти баланс между точностью и полнотой, что важно для практического применения модели в банковском маркетинге. В задаче регрессии существенное снижение ошибок прогнозирования стоимости автомобилей делает модель более полезной для реального использования.

Воспроизводимость и методология: Чёткое следование методологии исследования, фиксация случайных seed'ов и структурированный подход к проведению экспериментов обеспечили достоверность и воспроизводимость полученных результатов.

Таким образом, лабораторная работа успешно продемонстрировала весь цикл исследования алгоритма градиентного бустинга: от базовой реализации до улучшенных версий, с параллельной разработкой и валидацией собственной имплементации алгоритма. Полученные результаты подтверждают эффективность градиентного бустинга для решения реальных задач машинного обучения и демонстрируют важность правильного подхода к подготовке данных, настройке моделей и оценке их качества.

In [69]:
import pandas as pd

import pandas as pd


clf_results = [
    {
        "Model": "Sklearn GB — Baseline (Classification)",
        "Accuracy": metrics_base_clf['baseline_gb_accuracy'],
        "Precision": metrics_base_clf['baseline_gb_precision'],
        "Recall": metrics_base_clf['baseline_gb_recall'],
        "F1-score": metrics_base_clf['baseline_gb_f1'],
        "ROC-AUC": metrics_base_clf['baseline_gb_roc_auc'],
    },
    {
        "Model": "Sklearn GB — Improved (Classification)",
        "Accuracy": metrics_improved_clf['improved_gb_accuracy'],
        "Precision": metrics_improved_clf['improved_gb_precision'],
        "Recall": metrics_improved_clf['improved_gb_recall'],
        "F1-score": metrics_improved_clf['improved_gb_f1'],
        "ROC-AUC": metrics_improved_clf['improved_gb_roc_auc'],
    },
    {
        "Model": "Manual GB — Baseline (Classification)",
        "Accuracy": manual_baseline_acc,
        "Precision": manual_baseline_prec,
        "Recall": manual_baseline_rec,
        "F1-score": manual_baseline_f1,
        "ROC-AUC": manual_baseline_auc,
    },
    {
        "Model": "Manual GB — Improved (Classification)",
        "Accuracy": manual_improved_acc,
        "Precision": manual_improved_prec,
        "Recall": manual_improved_rec,
        "F1-score": manual_improved_f1,
        "ROC-AUC": manual_improved_auc,
    },
]

df_clf = pd.DataFrame(clf_results).round(4)

print("=== Classification models comparison ===")
display(df_clf)


reg_results = [
    {
        "Model": "Sklearn GB — Baseline (Regression)",
        "RMSE": metrics_base_reg['baseline_gb_rmse'],
        "MAE": metrics_base_reg['baseline_gb_mae'],
        "R2": metrics_base_reg['baseline_gb_r2'],
    },
    {
        "Model": "Sklearn GB — Improved (Regression)",
        "RMSE": metrics_improved_reg['improved_gb_rmse'],
        "MAE": metrics_improved_reg['improved_gb_mae'],
        "R2": metrics_improved_reg['improved_gb_r2'],
    },
    {
        "Model": "Manual GB — Baseline (Regression)",
        "RMSE": manual_baseline_rmse,
        "MAE": manual_baseline_mae,
        "R2": manual_baseline_r2,
    },
    {
        "Model": "Manual GB — Improved (Regression)",
        "RMSE": rmse_np(yte_r.values, pred_test),
        "MAE": mae_np(yte_r.values, pred_test),
        "R2": r2_np(yte_r.values, pred_test),
    },
]



df_reg = pd.DataFrame(reg_results)
df_reg = df_reg.round(4)

print("\n=== Regression models comparison ===")
display(df_reg)

=== Classification models comparison ===


,Model,Accuracy,Precision,Recall,F1-score,ROC-AUC
0,Sklearn GB — Baseline (Classification),0.9011,0.6728,0.2371,0.3506,0.8091
1,Sklearn GB — Improved (Classification),0.8860,0.4950,0.5851,0.5363,0.8164
2,Manual GB — Baseline (Classification),0.9042,0.6936,0.2683,0.3869,0.7946
3,Manual GB — Improved (Classification),0.8790,0.4708,0.6002,0.5277,0.8153



=== Regression models comparison ===


,Model,RMSE,MAE,R2
0,Sklearn GB — Baseline (Regression),6941.5369,4361.5179,0.9312
1,Sklearn GB — Improved (Regression),5118.4065,2812.8613,0.9626
2,Manual GB — Baseline (Regression),8092.3043,4934.3931,0.9065
3,Manual GB — Improved (Regression),5098.0809,2805.2063,0.9629
